In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.firefox.options import Options as foptions
from selenium.webdriver.edge.options import Options as eoptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from lxml import etree

Foptions = foptions()
# Foptions.headless = False

Eoptions = eoptions()
Eoptions.headless = True

# Veri setlerini yükle
unis_last = pd.read_excel('data/unis_last.xlsx')

try:
    say_df = pd.read_excel('data/say_df.xlsx')
    soz_df = pd.read_excel('data/soz_df.xlsx')
    ea_df = pd.read_excel('data/ea_df.xlsx')
    dil_df = pd.read_excel('data/dil_df.xlsx')
    tyt_df = pd.read_excel('data/tyt_df.xlsx')
    
    # Daha önce işlenmiş bölümleri topla
    processed_bolums = set()
    processed_bolums.update(say_df['bolum'].tolist())
    processed_bolums.update(soz_df['bolum'].tolist())
    processed_bolums.update(ea_df['bolum'].tolist())
    processed_bolums.update(dil_df['bolum'].tolist())
    processed_bolums.update(tyt_df['bolum'].tolist())
    
    print(f"Toplam {len(processed_bolums)} işlenmiş bölüm bulundu.")
    
except Exception as e:
    print(f"Veri dosyalarını yüklerken hata: {e}")
    print("İşlenmiş bölüm kaydı bulunamadı. Tüm bölümler işlenecek.")
    processed_bolums = set()
    
    # DataFrame'leri oluştur
    say_df = pd.DataFrame(columns=['yop','bolum','type','TYT Temel Matematik','TYT Fen Bilimleri','TYT Türkçe','TYT Sosyal Bilimler','AYT Matematik','AYT Fizik','AYT Kimya','AYT Biyoloji'])
    soz_df = pd.DataFrame(columns=['yop','bolum','type','TYT Temel Matematik','TYT Fen Bilimleri','TYT Türkçe','TYT Sosyal Bilimler','AYT Türk Dili ve Edebiyatı','AYT Coğrafya-1','AYT Coğrafya-2','AYT Tarih-1','AYT Tarih-2','AYT Felsefe Grubu','AYT Din Kültürü ve Ahlak Bilgisi'])
    ea_df = pd.DataFrame(columns=['yop','bolum','type','TYT Temel Matematik','TYT Fen Bilimleri','TYT Türkçe','TYT Sosyal Bilimler','AYT Türk Dili ve Edebiyatı','AYT Matematik','AYT Coğrafya-1','AYT Tarih-1'])
    dil_df = pd.DataFrame(columns=['yop','bolum','type','TYT Temel Matematik','TYT Fen Bilimleri','TYT Türkçe','TYT Sosyal Bilimler','AYT Yabancı Dil 1'])
    tyt_df = pd.DataFrame(columns=['yop','bolum','type','TYT Temel Matematik','TYT Fen Bilimleri','TYT Türkçe','TYT Sosyal Bilimler'])

# Ana döngü
for url in range(len(unis_last)):
    try:
        bolum = unis_last['bolum'][url]  # unis_last yapınıza göre düzenleyin
        
        # Eğer bu bölüm zaten işlenmişse atla
        if bolum in processed_bolums:
            print(f"'{bolum}' bölümü daha önce işlenmiş, atlıyorum.")
            continue
            
        driver = webdriver.Edge(options=Eoptions)
        driver.get(unis_last['url'][url])
        time.sleep(2)
        
        try:
            ec = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="mydata"]/thead/tr[1]/th[2]'))
            )
        except:
            print(f"'{bolum}' bölümü için tablo yüklenemedi, atlıyorum.")
            driver.quit()
            continue
            
        driver.maximize_window()
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        
        type = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div/h2/strong').text
        
        last_bracket_index = type.rfind('(')
        if last_bracket_index != -1:  # Eğer parantez bulunduysa
            closing_bracket_index = type.rfind(')')
            if closing_bracket_index > last_bracket_index:  # Eğer kapanan parantez, açılan parantezden sonra geliyorsa
                # Parantezlerin içindeki metni al (parantezler hariç)
                type_in_brackets = type[last_bracket_index+1:closing_bracket_index]
        else:
            type_in_brackets = "Bilinmiyor"

        # SÖZ alanı için
        if type_in_brackets == 'SÖZ':
            try:
                rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                
                if len(rows) == 0 or (len(rows) == 1 and "Tabloda herhangi bir veri mevcut değil" in rows[0].text):
                    print(f"'{bolum}' bölümü için Tabloda herhangi bir veri mevcut değil, atlıyorum.")
                    continue
                
                has_next_page = True
                
                while has_next_page:
                    rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                    row_count = len(rows)
                    
                    for x in range(1, row_count + 1):
                        try:
                            uni_name = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[2]/small/a').get_attribute('href').split('=')[1].strip()
                            tyt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[10]').text
                            tyt_fen = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[11]').text
                            tyt_turkce = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[8]').text
                            tyt_sosyal = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[9]').text
                            ayt_edeb = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[12]').text
                            ayt_coğ1 = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[14]').text
                            ayt_coğ2 = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[16]').text
                            ayt_tarih1 = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[13]').text
                            ayt_tarih2 = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[15]').text
                            ayt_felsefe = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[17]').text
                            ayt_din = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[18]').text
                            
                            # Pandas yeni versiyonlarda append yerine concat kullanıyor
                            new_row = pd.DataFrame([{
                                'yop': uni_name,
                                'bolum': bolum,
                                'type': type_in_brackets,
                                'TYT Temel Matematik': tyt_mat,
                                'TYT Fen Bilimleri': tyt_fen,
                                'TYT Türkçe': tyt_turkce,
                                'TYT Sosyal Bilimler': tyt_sosyal,
                                'AYT Türk Dili ve Edebiyatı': ayt_edeb,
                                'AYT Coğrafya-1': ayt_coğ1,
                                'AYT Coğrafya-2': ayt_coğ2,
                                'AYT Tarih-1': ayt_tarih1,
                                'AYT Tarih-2': ayt_tarih2,
                                'AYT Felsefe Grubu': ayt_felsefe,
                                'AYT Din Kültürü ve Ahlak Bilgisi': ayt_din
                            }])
                            
                            soz_df = pd.concat([soz_df, new_row], ignore_index=True)
                            
                        except Exception as e:
                            print(f"SÖZ - Satır {x} işlenirken hata: {e}")
                    
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, '#mydata_next')
                        if 'disabled' not in next_button.get_attribute('class'):
                            next_button.click()
                            time.sleep(2)
                        else:
                            has_next_page = False
                    except:
                        has_next_page = False
            
            except Exception as e:
                print(f"SÖZ bölümü işleme hatası: {e}")

        # SAY alanı için
        elif type_in_brackets == 'SAY':
            try:
                rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                
                if len(rows) == 0 or (len(rows) == 1 and "Tabloda herhangi bir veri mevcut değil" in rows[0].text):
                    print(f"'{bolum}' bölümü için Tabloda herhangi bir veri mevcut değil, atlıyorum.")
                    continue
                
                has_next_page = True
                
                while has_next_page:
                    rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                    row_count = len(rows)
                    
                    for x in range(1, row_count + 1):
                        try:
                            uni_name = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[2]/small/a').get_attribute('href').split('=')[1].strip()
                            tyt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[10]').text
                            tyt_fen = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[11]').text
                            tyt_turkce = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[8]').text
                            tyt_sosyal = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[9]').text
                            ayt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[12]').text
                            ayt_fizik = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[13]').text
                            ayt_kimya = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[14]').text
                            ayt_biyoloji = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[15]').text
                            
                            new_row = pd.DataFrame([{
                                'yop': uni_name,
                                'bolum': bolum,
                                'type': type_in_brackets,
                                'TYT Temel Matematik': tyt_mat,
                                'TYT Fen Bilimleri': tyt_fen,
                                'TYT Türkçe': tyt_turkce,
                                'TYT Sosyal Bilimler': tyt_sosyal,
                                'AYT Matematik': ayt_mat,
                                'AYT Fizik': ayt_fizik,
                                'AYT Kimya': ayt_kimya,
                                'AYT Biyoloji': ayt_biyoloji
                            }])
                            
                            say_df = pd.concat([say_df, new_row], ignore_index=True)
                            
                        except Exception as e:
                            print(f"SAY - Satır {x} işlenirken hata: {e}")
                    
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, '#mydata_next')
                        if 'disabled' not in next_button.get_attribute('class'):
                            next_button.click()
                            time.sleep(2)
                        else:
                            has_next_page = False
                    except:
                        has_next_page = False
            
            except Exception as e:
                print(f"SAY bölümü işleme hatası: {e}")

        # EA alanı için
        elif type_in_brackets == 'EA':
            try:
                rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                
                if len(rows) == 0 or (len(rows) == 1 and "Tabloda herhangi bir veri mevcut değil" in rows[0].text):
                    print(f"'{bolum}' bölümü için Tabloda herhangi bir veri mevcut değil, atlıyorum.")
                    continue
                
                has_next_page = True
                
                while has_next_page:
                    rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                    row_count = len(rows)
                    
                    for x in range(1, row_count + 1):
                        try:
                            uni_name = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[2]/small/a').get_attribute('href').split('=')[1].strip()
                            tyt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[10]').text
                            tyt_fen = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[11]').text
                            tyt_turkce = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[8]').text
                            tyt_sosyal = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[9]').text
                            ayt_edeb = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[13]').text
                            ayt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[12]').text
                            ayt_coğ1 = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[15]').text
                            ayt_tarih1 = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[14]').text
                            
                            new_row = pd.DataFrame([{
                                'yop': uni_name,
                                'bolum': bolum,
                                'type': type_in_brackets,
                                'TYT Temel Matematik': tyt_mat,
                                'TYT Fen Bilimleri': tyt_fen,
                                'TYT Türkçe': tyt_turkce,
                                'TYT Sosyal Bilimler': tyt_sosyal,
                                'AYT Türk Dili ve Edebiyatı': ayt_edeb,
                                'AYT Matematik': ayt_mat,
                                'AYT Coğrafya-1': ayt_coğ1,
                                'AYT Tarih-1': ayt_tarih1
                            }])
                            
                            ea_df = pd.concat([ea_df, new_row], ignore_index=True)
                            
                        except Exception as e:
                            print(f"EA - Satır {x} işlenirken hata: {e}")
                    
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, '#mydata_next')
                        if 'disabled' not in next_button.get_attribute('class'):
                            next_button.click()
                            time.sleep(2)
                        else:
                            has_next_page = False
                    except:
                        has_next_page = False
            
            except Exception as e:
                print(f"EA bölümü işleme hatası: {e}")

        # DİL alanı için
        elif type_in_brackets == 'DİL':
            try:
                rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                
                if len(rows) == 0 or (len(rows) == 1 and "Tabloda herhangi bir veri mevcut değil" in rows[0].text):
                    print(f"'{bolum}' bölümü için Tabloda herhangi bir veri mevcut değil, atlıyorum.")
                    continue
                
                has_next_page = True
                
                while has_next_page:
                    rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                    row_count = len(rows)
                    
                    for x in range(1, row_count + 1):
                        try:
                            uni_name = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[2]/small/a').get_attribute('href').split('=')[1].strip()
                            tyt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[10]').text
                            tyt_fen = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[11]').text
                            tyt_turkce = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[8]').text
                            tyt_sosyal = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[9]').text
                            ayt_dil1 = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[12]').text
                            
                            new_row = pd.DataFrame([{
                                'yop': uni_name,
                                'bolum': bolum,
                                'type': type_in_brackets,
                                'TYT Temel Matematik': tyt_mat,
                                'TYT Fen Bilimleri': tyt_fen,
                                'TYT Türkçe': tyt_turkce,
                                'TYT Sosyal Bilimler': tyt_sosyal,
                                'AYT Yabancı Dil 1': ayt_dil1
                            }])
                            
                            dil_df = pd.concat([dil_df, new_row], ignore_index=True)
                            
                        except Exception as e:
                            print(f"DİL - Satır {x} işlenirken hata: {e}")
                    
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, '#mydata_next')
                        if 'disabled' not in next_button.get_attribute('class'):
                            next_button.click()
                            time.sleep(2)
                        else:
                            has_next_page = False
                    except:
                        has_next_page = False
            
            except Exception as e:
                print(f"DİL bölümü işleme hatası: {e}")

        # TYT alanı için
        elif 'TYT' in type_in_brackets:
            time.sleep(2)
            try:
                rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                
                if len(rows) == 0 or (len(rows) == 1 and "Tabloda herhangi bir veri mevcut değil" in rows[0].text):
                    print(f"'{bolum}' bölümü için Tabloda herhangi bir veri mevcut değil, atlıyorum.")
                    continue
                
                has_next_page = True
                
                while has_next_page:
                    rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                    row_count = len(rows)
                    
                    for x in range(1, row_count + 1):
                        try:
                            uni_name = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[2]/small/a').get_attribute('href').split('=')[1].strip()
                            tyt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[10]').text
                            tyt_fen = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[11]').text
                            tyt_turkce = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[8]').text
                            tyt_sosyal = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[9]').text
                            
                            new_row = pd.DataFrame([{
                                'yop': uni_name,
                                'bolum': bolum,
                                'type': type_in_brackets,
                                'TYT Temel Matematik': tyt_mat,
                                'TYT Fen Bilimleri': tyt_fen,
                                'TYT Türkçe': tyt_turkce,
                                'TYT Sosyal Bilimler': tyt_sosyal
                            }])
                            
                            tyt_df = pd.concat([tyt_df, new_row], ignore_index=True)
                            
                        except Exception as e:
                            print(f"TYT - Satır {x} işlenirken hata: {e}")
                    
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, '#mydata_next')
                        if 'disabled' not in next_button.get_attribute('class'):
                            next_button.click()
                            time.sleep(2)
                        else:
                            has_next_page = False
                    except:
                        has_next_page = False
            
            except Exception as e:
                print(f"TYT bölümü işleme hatası: {e}")
        
        else:
            print(f"'{bolum}' bölümü için geçerli puan türü bulunamadı: {type_in_brackets}")
                
        driver.quit()
        
        # Her 10 URL'de bir veri kaydet
        if url % 10 == 0 and url > 0:
        #     say_df.to_excel("data/say_df.xlsx", index=False)
        #     soz_df.to_excel("data/soz_df.xlsx", index=False)
        #     ea_df.to_excel("data/ea_df.xlsx", index=False)
        #     dil_df.to_excel("data/dil_df.xlsx", index=False)
            tyt_df.to_excel("data/tyt_df.xlsx", index=False)
            print(f"{url} bölüm işlendi, veriler geçici olarak kaydedildi.")
            
    except Exception as e:
        print(f"URL {url} işlenirken hata oluştu: {e}")
        continue

# # Tüm işlem bittikten sonra verileri kaydet
# say_df.to_excel("data/say_df.xlsx", index=False)
# soz_df.to_excel("data/soz_df.xlsx", index=False)
# ea_df.to_excel("data/ea_df.xlsx", index=False)
# dil_df.to_excel("data/dil_df.xlsx", index=False)
tyt_df.to_excel("data/tyt_df.xlsx", index=False)

print("Tüm veriler başarıyla toplandı ve kaydedildi.")

C:\Users\batuh\AppData\Local\Temp\ipykernel_14896\2720831019.py:16: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  Eoptions.headless = True


Toplam 468 işlenmiş bölüm bulundu.
'ABAZA DİLİ VE EDEBİYATI' bölümü daha önce işlenmiş, atlıyorum.
'ACİL YARDIM VE AFET YÖNETİMİ (FAKÜLTE)' bölümü daha önce işlenmiş, atlıyorum.
'ACİL YARDIM VE AFET YÖNETİMİ (YÜKSEKOKUL)' bölümü daha önce işlenmiş, atlıyorum.
'ADLİ BİLİMLER' bölümü daha önce işlenmiş, atlıyorum.
'ADLİ BİLİŞİM MÜHENDİSLİĞİ' bölümü daha önce işlenmiş, atlıyorum.
'ADLİ BİLİŞİM MÜHENDİSLİĞİ (M.T.O.K.)' bölümü daha önce işlenmiş, atlıyorum.
'AĞAÇ İŞLERİ ENDÜSTRİ MÜHENDİSLİĞİ' bölümü daha önce işlenmiş, atlıyorum.
'AĞAÇ İŞLERİ ENDÜSTRİ MÜHENDİSLİĞİ (M.T.O.K.)' bölümü daha önce işlenmiş, atlıyorum.
'AİLE VE TÜKETİCİ BİLİMLERİ' bölümü daha önce işlenmiş, atlıyorum.
'AKTÜERYA BİLİMLERİ' bölümü daha önce işlenmiş, atlıyorum.
'ALMAN DİLİ VE EDEBİYATI' bölümü daha önce işlenmiş, atlıyorum.
'ALMANCA MÜTERCİM VE TERCÜMANLIK' bölümü daha önce işlenmiş, atlıyorum.
'ALMANCA ÖĞRETMENLİĞİ' bölümü daha önce işlenmiş, atlıyorum.
'AMERİKAN KÜLTÜRÜ VE EDEBİYATI' bölümü daha önce işlenmiş, at